In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris


data = load_iris()
type(data)

sklearn.utils._bunch.Bunch